Python script to scrape player in-game statistics data from fbref.com and merge with player valuations. Extracted data from 10 professional soccer leagues around the world.

In [1]:
# libraries
import pandas as pd
import numpy as np
import requests
import time
import random
import fuzzywuzzy as fuzz

years_special = ['2022-2023']
numbers = [11,20,9,12,30,13,40,37,23,32]
stats = ['passing','defense','shooting','playingtime']
leagues = ['-Serie-A-Stats','-Bundesliga-Stats',
           '-Premier-League-Stats','-La-Liga-Stats','-Russian-Premier-League-Stats',
           '-Ligue-1-Stats','-Scottish-Premiership-Stats','-Belgian-Pro-League-Stats','-Eredivisie-Stats',
           '-Primeira-Liga-Stats']
           
league_names = ['Serie A','Bundesliga','Premier League','La Liga', 
               'Premier Liga', 'Ligue 1', 'Scottish Premiership',
               'Jupiler Pro League', 'Eredivisie', 'Liga Nos']
                
shot = []
defense = []
passing = []
playingtime = []
count = 0
for n in range(len(numbers)):
    for t in stats:
        for z in years_special:
            data = 'https://fbref.com/en/comps/' + str(numbers[count]) + '/' + z + '/' + t + '/' + z + leagues[count]
            response = requests.get(data).text.replace('<!--', '').replace('-->', '')
            df = pd.read_html(response, header=1)[2]
            df['Based'] = league_names[count]
            if t == 'passing':
                passing.append(df)
            elif t == 'defense':
                defense.append(df)
            elif t == 'shooting':
                shot.append(df)
            else:
                playingtime.append(df)
            time.sleep(3)
            print('Imported ' + t + ' data for ' + league_names[count])

    count += 1

Imported passing data for Serie A
Imported defense data for Serie A
Imported shooting data for Serie A
Imported playingtime data for Serie A
Imported passing data for Bundesliga
Imported defense data for Bundesliga
Imported shooting data for Bundesliga
Imported playingtime data for Bundesliga
Imported passing data for Premier League
Imported defense data for Premier League
Imported shooting data for Premier League
Imported playingtime data for Premier League
Imported passing data for La Liga
Imported defense data for La Liga
Imported shooting data for La Liga
Imported playingtime data for La Liga
Imported passing data for Premier Liga
Imported defense data for Premier Liga
Imported shooting data for Premier Liga
Imported playingtime data for Premier Liga
Imported passing data for Ligue 1
Imported defense data for Ligue 1
Imported shooting data for Ligue 1
Imported playingtime data for Ligue 1
Imported passing data for Scottish Premiership
Imported defense data for Scottish Premiership


In [2]:
for k in range(4):
    
    if stats[k] == 'passing':
        count = 0
        for df in passing:
            # rename columns
            df.rename(columns={'Cmp':'Total Cmp', 'Att': 'Total Att', 'Cmp%':'Total Cmp%','Cmp.1':'Short Cmp', 
                               'Att.1':'Short Att','Cmp%.1':'Short Cmp%', 'Cmp.2':'Medium Cmp', 'Att.2':'Medium Att',
                               'Cmp%.2':'Medium Cmp%', 'Cmp.3':'Long Cmp', 'Att.3':'Long Att', 'Cmp%.3':'Long Cmp%'}, inplace = True)
            
            df = df.drop(columns=['Rk','Nation','Pos','Matches','A-xAG','Short Cmp','Medium Cmp','Long Cmp',
                                  'Short Att','Medium Att','Long Att','Total Cmp','Total Att'])
            passing[count] = df
            count += 1


    elif stats[k] == 'defense':
        count = 0
        for df in defense:

            # rename columns
            df['Age'] = df['Age'].str[:2]
            df['Position'] = df['Pos'].str[:2]
            df['Position'] = df['Position'].replace({'MF': 'Midfielder', 'DF': 'Defender', 'FW': 'Forward', 'GK': 'Goalkeeper'})

            df = df.drop(columns=['Rk','Nation','Pos','Matches'])

            df.rename(columns={'Sh':'Blocks Sh','Pass':'Blocks Pass','Tkl.1':'drTkl'}, inplace = True)
            
            defense[count] = df
            count += 1
            
    elif stats[k] == 'playingtime':
        count = 0
        for df in playingtime:

            # rename columns
            df['Age'] = df['Age'].str[:2]

            df = df.drop(columns=['Rk','Nation','Pos','Matches','MP','Min','Mn/MP','Min%','Starts',
                                  'Mn/Start','Compl','Subs','Mn/Sub','unSub'])
            df.rename(columns={'On-Off.1':'xG_On-Off'}, inplace = True)

            playingtime[count] = df
            count += 1

    else:
        count = 0
        for df in shot:

            # rename columns
            df['Age'] = df['Age'].str[:2]

            df = df.drop(columns=['Rk','Nation','Pos','Matches'])
            
            shot[count] = df
            count += 1
            
    df_shot = pd.concat(shot)
    df_def = pd.concat(defense)
    df_pass = pd.concat(passing)
    df_pt = pd.concat(playingtime)

final = pd.merge(pd.merge(pd.merge(df_shot,df_def,on=['Player','Squad','Born','Based','Age','90s']),
                 df_pass,on=['Player','Squad','Born','Based','Age','90s']),
                 df_pt,on=['Player','Squad','Born','Based','Age','90s'])

final['Born'] = final['Born'].astype(str)
final['Born'] = final['Born'].str[-2:]
final['player_code'] = final['Player'].str.lower().str.replace(' ','-') + final['Born'].astype(str)

replacements = {
    'à': 'a', 'á': 'a', 'â': 'a', 'ä': 'a', 'ǎ': 'a', 'æ': 'a', 'ã': 'a', 'å': 'a', 'ā': 'a',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e', 'ě': 'e', 'ẽ': 'e', 'ē': 'e', 'ė': 'e', 'ę': 'e',
    'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i', 'ǐ': 'i', 'ĩ': 'i', 'ī': 'i', 'ı': 'i', 'į': 'i',
    'ò': 'o', 'ó': 'o', 'ô': 'o', 'ö': 'o', 'ǒ': 'o', 'œ': 'o', 'ø': 'o', 'õ': 'o', 'ō': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u', 'ǔ': 'u', 'ũ': 'u', 'ū': 'u', 'ű': 'u', 'ů': 'u',
    'ğ':'g','ġ':'g','Ğ':'G','Ġ':'G','ç':'c','ć':'c','č':'c','ċ':'c','ł':'l','ļ':'l','ľ':'l',
    'ß':'s','ş':'s','ș':'s','ś':'s','š':'s','ķ':'k','ñ':'n','ń':'n','ņ':'n','ň':'n','ŵ':'w',
    'ź':'z','ž':'z','ż':'z','þ':'b','đ':'d',"'":'',"’":'','ý':'y','i̇':'i','ă':'a','ř':'r'
}

for key in replacements:
    final['player_code'] = final['player_code'].str.replace(key, replacements[key])
    
final.to_csv('player_data_2023.csv')
final = final[final['Position'] != 'Goalkeeper']

In [3]:
final[final.duplicated(['player_code','Born'], keep=False)]

,Player,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,...,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,xG_On-Off,player_code
1,Oliver Abildgaard,Hellas Verona,26,96,6.6,0,3,2,66.7,0.46,...,11,-9,-1.37,-0.76,4.9,9.3,-4.4,-0.67,-0.36,oliver-abildgaard96
15,Bruno Amione,Hellas Verona,20,02,0.6,0,0,0,NaN,0.00,...,3,-1,-1.55,-0.83,0.9,1.7,-0.8,-1.27,-0.91,bruno-amione02
16,Bruno Amione,Sampdoria,20,02,23.2,1,12,2,16.7,0.52,...,41,-24,-1.04,+0.52,21.7,39.8,-18.0,-0.78,+0.18,bruno-amione02
33,Nedim Bajrami,Empoli,23,99,10.4,1,21,7,33.3,2.01,...,14,-6,-0.57,-0.36,8.9,18.6,-9.7,-0.93,-0.58,nedim-bajrami99
34,Nedim Bajrami,Sassuolo,23,99,9.1,1,23,8,34.8,2.53,...,19,-10,-1.10,-0.96,11.7,11.9,-0.2,-0.02,-0.09,nedim-bajrami99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5377,Jan Vertonghen,Benfica,35,87,0.0,0,0,0,NaN,0.00,...,0,0,0.00,-1.82,0.0,0.0,0.0,0.00,-1.89,jan-vertonghen87
5383,Vitinha,Braga,22,00,12.2,7,48,20,41.7,3.94,...,9,+19,+1.56,+0.37,21.3,12.9,+8.4,+0.69,+0.19,vitinha00
5391,Julian Weigl,Benfica,26,95,0.6,0,1,0,0.0,1.55,...,0,+1,+1.55,-0.28,1.0,0.0,+1.0,+1.52,-0.37,julian-weigl95
5394,Willyan,Portimonense,27,95,4.0,0,4,2,50.0,1.00,...,2,+4,+1.00,+1.90,5.5,3.3,+2.2,+0.56,+1.18,willyan95


In [4]:
final[final['player_code'].str.contains('bereszynski')]

,Player,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,...,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,xG_On-Off,player_code
62,Bartosz Bereszyński,Sampdoria,30,92,13.3,0,3,0,0.0,0.23,...,20,-12,-0.90,+0.52,9.2,18.8,-9.6,-0.72,+0.20,bartosz-bereszynski92
63,Bartosz Bereszyński,Napoli,30,92,1.8,0,0,0,NaN,0.00,...,4,-1,-0.54,-1.93,2.1,3.3,-1.2,-0.64,-1.59,bartosz-bereszynski92


In [5]:
pv = pd.read_csv('transfermarkt_data.csv')
pv['Born']=pv['date_of_birth'].str[-2:].astype(str)

merged = pd.merge(final,pv,on=['player_code','Squad','Born','Based']).drop(['current_club_domestic_competition_id','first_name',
                                                             'last_name','player_club_domestic_competition_id',
                                                    'club_id','last_season','current_club_name','club_code','date',
                                                                    'current_club_id','player_id','Unnamed: 0','name',
                                                                   'date_of_birth'], axis=1)
merged

,Player,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,...,xG+/-,xG+/-90,xG_On-Off,player_code,country_of_birth,sub_position,foot,height_in_cm,contract_expiration_date,market_value_in_eur
0,James Abankwah,Udinese,18,04,0.7,0,0,0,NaN,0.00,...,-0.3,-0.37,-0.25,james-abankwah04,Ireland,Centre-Back,right,182.0,6/30/26 0:00,200000
1,Oliver Abildgaard,Hellas Verona,26,96,6.6,0,3,2,66.7,0.46,...,-4.4,-0.67,-0.36,oliver-abildgaard96,Denmark,Defensive Midfield,left,192.0,6/30/23 0:00,5000000
2,Tammy Abraham,Roma,24,97,24.3,8,62,28,45.2,2.55,...,+14.8,+0.61,-0.16,tammy-abraham97,England,Centre-Forward,right,194.0,6/30/26 0:00,45000000
3,Christian Acella,Cremonese,20,02,0.2,0,0,0,NaN,0.00,...,-0.1,-0.65,+0.07,christian-acella02,Italy,Central Midfield,right,NaN,NaN,150000
4,Francesco Acerbi,Inter,34,88,26.9,0,18,2,11.1,0.67,...,+22.9,+0.85,+0.07,francesco-acerbi88,Italy,Centre-Back,left,192.0,6/30/23 0:00,4000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,Xadas,Marítimo,24,97,26.3,3,74,26,35.1,2.81,...,-10.0,-0.38,-0.01,xadas97,Portugal,Attacking Midfield,left,179.0,6/30/24 0:00,1500000
3946,Moses Yaw,Arouca,23,99,3.5,0,1,1,100.0,0.29,...,-4.5,-1.32,-1.11,moses-yaw99,NaN,Central Midfield,right,170.0,6/30/24 0:00,200000
3947,Zaydou Youssouf,Famalicão,23,99,23.8,2,24,6,25.0,1.01,...,-2.0,-0.08,+0.75,zaydou-youssouf99,France,Central Midfield,left,182.0,6/30/26 0:00,3000000
3948,Kévin Zohi,Vizela,25,96,10.0,2,26,9,34.6,2.60,...,+2.1,+0.21,+0.28,kevin-zohi96,Cote d'Ivoire,Right Winger,right,180.0,6/30/24 0:00,800000


In [6]:
merged.to_csv('nongk_data_2023.csv')

In [7]:
merged.columns

Index(['Player', 'Squad', 'Age', 'Born', '90s', 'Gls', 'Sh', 'SoT', 'SoT%',
       'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG',
       'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Based', 'Tkl', 'TklW', 'Def 3rd',
       'Mid 3rd', 'Att 3rd', 'drTkl', 'Att', 'Tkl%', 'Lost', 'Blocks',
       'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Position',
       'Total Cmp%', 'TotDist', 'PrgDist', 'Short Cmp%', 'Medium Cmp%',
       'Long Cmp%', 'Ast', 'xAG', 'xA', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP',
       'PPM', 'onG', 'onGA', '+/-', '+/-90', 'On-Off', 'onxG', 'onxGA',
       'xG+/-', 'xG+/-90', 'xG_On-Off', 'player_code', 'country_of_birth',
       'sub_position', 'foot', 'height_in_cm', 'contract_expiration_date',
       'market_value_in_eur'],
      dtype='object')

In [8]:
tr = pd.read_csv('2022_transfers.csv')
bleh = pd.merge(merged,tr,on=['player_code','Squad']).drop(['Unnamed: 0','age','contract_expiration_date','player_name','season'],axis=1)
bleh

,Player,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,...,xG+/-90,xG_On-Off,player_code,country_of_birth,sub_position,foot,height_in_cm,market_value_in_eur,fee,transfer_period
0,Oliver Abildgaard,Hellas Verona,26,96,6.6,0,3,2,66.7,0.46,...,-0.67,-0.36,oliver-abildgaard96,Denmark,Defensive Midfield,left,192.0,5000000,0.0,Winter
1,Francesco Acerbi,Inter,34,88,26.9,0,18,2,11.1,0.67,...,+0.85,+0.07,francesco-acerbi88,Italy,Centre-Back,left,192.0,4000000,0.0,Summer
2,Michel Aebischer,Bologna,25,97,17.1,1,15,2,13.3,0.88,...,-0.28,-0.28,michel-aebischer97,Switzerland,Central Midfield,right,183.0,3500000,4000000.0,Summer
3,Felix Afena-Gyan,Cremonese,19,03,9.1,0,21,2,9.5,2.31,...,-0.52,+0.26,felix-afena-gyan03,Ghana,Centre-Forward,right,175.0,5500000,6000000.0,Summer
4,Kevin Agudelo,Spezia,23,98,23.7,0,28,7,25.0,1.18,...,-0.60,-0.35,kevin-agudelo98,Colombia,Attacking Midfield,left,178.0,3500000,2700000.0,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,Steven Vitória,Chaves,35,87,29.4,7,24,6,25.0,0.82,...,-0.29,+0.02,steven-vitoria87,Canada,Centre-Back,right,195.0,100000,0.0,Summer
1395,Ilija Vukotić,Boavista,23,99,4.5,0,2,1,50.0,0.45,...,-0.29,-0.10,ilija-vukotic99,Yugoslavia (Republic),Defensive Midfield,left,191.0,400000,0.0,Summer
1396,Weverson,Arouca,22,00,0.8,0,0,0,NaN,0.00,...,+0.22,+0.54,weverson00,Brazil,Left-Back,left,185.0,900000,0.0,Winter
1397,Roan Wilson,Gil Vicente FC,20,02,1.3,0,1,0,0.0,0.77,...,+1.08,+1.41,roan-wilson02,Costa Rica,Central Midfield,right,184.0,250000,0.0,Winter


In [9]:
squ = pd.read_csv('squad_comparisons.csv')
oop = pd.merge(bleh,squ,on=['Squad'])
oop[['+/-90','90s']] = oop[['+/-90','90s']].astype(float)
oop['Position'].unique()

array(['Midfielder', 'Defender', 'Forward'], dtype=object)

In [10]:
# Define conditions and corresponding calculations
conditions = [
    (oop["Position"] == "Forward"),
    (oop["Position"] == "Midfielder"),
    (oop["Position"] == "Defender")
]

calculations = [
    (oop['+/-90'] + oop['sq_Gls/90'])*oop['90s'],
    (oop['+/-90'] + oop['sq_Gls/90'] - oop['opp_sq_Gls/90'])*oop['90s'],
    (oop['+/-90'] - oop['opp_sq_Gls/90'])*oop['90s'],
]

# Apply conditions and calculations to create the "Target" column
oop['Target'] = np.select(conditions, calculations, default=np.nan)
oop

,Player,Squad,Age,Born,90s,Gls,Sh,SoT,SoT%,Sh/90,...,sq_Gls,sq_Ast,sq_Gls/90,sq_Ast/90,opp_sq_Poss,opp_sq_Gls,opp_sq_Ast,opp_sq_Gls/90,opp_sq_Ast/90,Target
0,Oliver Abildgaard,Hellas Verona,26,96,6.6,0,3,2,66.7,0.46,...,-34.0,-22.0,-0.90,-0.58,9.4,-1.0,10.0,-0.03,0.26,-14.784
1,Jayden Braaf,Hellas Verona,19,02,3.2,0,9,3,33.3,2.81,...,-34.0,-22.0,-0.90,-0.58,9.4,-1.0,10.0,-0.03,0.26,-4.768
2,Juan David Cabal,Hellas Verona,21,01,4.9,0,1,1,100.0,0.20,...,-34.0,-22.0,-0.90,-0.58,9.4,-1.0,10.0,-0.03,0.26,-7.791
3,Fabio Depaoli,Hellas Verona,25,97,24.1,2,19,4,21.1,0.79,...,-34.0,-22.0,-0.90,-0.58,9.4,-1.0,10.0,-0.03,0.26,-18.316
4,Josh Doig,Hellas Verona,20,02,15.0,2,18,6,33.3,1.20,...,-34.0,-22.0,-0.90,-0.58,9.4,-1.0,10.0,-0.03,0.26,-8.550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,Monsuru Opeyemi,Vizela,18,03,0.5,0,1,1,100.0,2.00,...,-6.0,1.0,-0.18,0.03,-0.9,-19.0,-6.0,-0.56,-0.18,-0.090
1336,Pedro Ortiz,Vizela,21,00,5.3,0,2,0,0.0,0.37,...,-6.0,1.0,-0.18,0.03,-0.9,-19.0,-6.0,-0.56,-0.18,7.950
1337,Milutin Osmajić,Vizela,23,99,22.7,8,46,22,47.8,2.03,...,-6.0,1.0,-0.18,0.03,-0.9,-19.0,-6.0,-0.56,-0.18,-7.037
1338,Matheus Pereira,Vizela,21,00,9.0,1,2,1,50.0,0.22,...,-6.0,1.0,-0.18,0.03,-0.9,-19.0,-6.0,-0.56,-0.18,8.010


In [11]:
test = oop[oop['Player']=='Romelu Lukaku']
test[['+/-90', 'sq_Gls/90', 'opp_sq_Gls/90','Target','Position']]

,+/-90,sq_Gls/90,opp_sq_Gls/90,Target,Position
21,0.6,-0.39,0.21,3.864,Forward


In [12]:
nongk_merger = oop.drop(['sq_Poss','sq_Gls','sq_Ast','sq_Gls/90','sq_Ast/90','opp_sq_Poss',
 'opp_sq_Gls','opp_sq_Ast','opp_sq_Gls/90','opp_sq_Ast/90'], axis=1)

In [13]:
nongk_merger.to_csv('final_nongk_data.csv')

In [14]:
list(nongk_merger.columns)

['Player',
 'Squad',
 'Age',
 'Born',
 '90s',
 'Gls',
 'Sh',
 'SoT',
 'SoT%',
 'Sh/90',
 'SoT/90',
 'G/Sh',
 'G/SoT',
 'Dist',
 'FK',
 'PK',
 'PKatt',
 'xG',
 'npxG',
 'npxG/Sh',
 'G-xG',
 'np:G-xG',
 'Based',
 'Tkl',
 'TklW',
 'Def 3rd',
 'Mid 3rd',
 'Att 3rd',
 'drTkl',
 'Att',
 'Tkl%',
 'Lost',
 'Blocks',
 'Blocks Sh',
 'Blocks Pass',
 'Int',
 'Tkl+Int',
 'Clr',
 'Err',
 'Position',
 'Total Cmp%',
 'TotDist',
 'PrgDist',
 'Short Cmp%',
 'Medium Cmp%',
 'Long Cmp%',
 'Ast',
 'xAG',
 'xA',
 'KP',
 '1/3',
 'PPA',
 'CrsPA',
 'PrgP',
 'PPM',
 'onG',
 'onGA',
 '+/-',
 '+/-90',
 'On-Off',
 'onxG',
 'onxGA',
 'xG+/-',
 'xG+/-90',
 'xG_On-Off',
 'player_code',
 'country_of_birth',
 'sub_position',
 'foot',
 'height_in_cm',
 'market_value_in_eur',
 'fee',
 'transfer_period',
 'Target']